In [30]:
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from dataclasses import dataclass

In [31]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


### Prepare the data

In [33]:
class ImpurityDataset(Dataset):
    
    def __init__(self, dataframe, fixed_features, labels, feature_scaler=None, label_r_scaler=None, label_i_scaler=None, device=None):
        assert len(labels) % 2 == 0
        
        self.fixed_features = fixed_features
        self.labels = labels
        self.n_samples = len(dataframe)
        
        self.output_length = 2
        self.input_length = len(fixed_features) + len(labels) - self.output_length
        self.sequence_length = len(labels) // self.output_length

        df_features = dataframe[fixed_features]
        df_labels = dataframe[labels]
        
        if feature_scaler is not None and label_r_scaler is not None and label_i_scaler is not None:
            xs = feature_scaler.transform(df_features)
            #ys = df_labels.values
            
            ys_r = label_r_scaler.transform(df_labels[df_labels.columns[::2]])
            ys_i = label_i_scaler.transform(df_labels[df_labels.columns[1::2]])
            
            ys = np.empty((ys_r.shape[0], ys_r.shape[1] * 2), dtype=ys_r.dtype)
            ys[:, ::2] = ys_r  # fill even indices with real parts
            ys[:, 1::2] = ys_i  # fill odd indices with imaginary parts
        else:
            xs = df_features.values
            ys = df_labels.values

        feature_data = np.zeros((self.n_samples, self.sequence_length, self.input_length))
        label_data = np.zeros((self.n_samples, self.sequence_length, self.output_length))
        
        for i in range(self.n_samples):
            for j in range(self.sequence_length):
                xi = xs[i]
                xj = ys[i][0:j*2]
                y = ys[i][j*2:j*2+2]
        
                features = np.concatenate([xi, xj], axis=0)
                pad_width = self.input_length - len(features)
                feature_data[i, j, :] = np.pad(features, (0, pad_width))
                label_data[i, j, :] = y

        self.feature_data = torch.tensor(feature_data, dtype=torch.float).to(device)
        self.label_data = torch.tensor(label_data, dtype=torch.float).to(device)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

In [34]:
def compute_scalers(dataframe, fixed_features, labels, test_size=0.1, random_state=None):
    train_df, _ = train_test_split(dataframe, test_size=test_size, random_state=random_state)
    df_features = train_df[fixed_features]
    df_labels = train_df[labels]
    
    feature_scaler = StandardScaler()
    label_r_scaler = StandardScaler()
    label_i_scaler = StandardScaler()

    feature_scaler.fit(df_features)
    label_r_scaler.fit(df_labels[df_labels.columns[::2]]) # get only real columns
    label_i_scaler.fit(df_labels[df_labels.columns[1::2]]) # get only imaginary columns

    return feature_scaler, label_r_scaler, label_i_scaler

In [35]:
file_path = '../data/20230825_144318_10k_EVDoubExp-TExp-wmax5-sparse-hyb_with_perturbation.csv'

#fixed_features = ['beta', 'U', 'Eimp', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']
fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3']
labels = ['ReSf1', 'ImSf1', 'ReSf3', 'ImSf3', 'ReSf5', 'ImSf5', 'ReSf7', 'ImSf7', 'ReSf9', 'ImSf9', 'ReSf11', 'ImSf11', 'ReSf13', 'ImSf13', 'ReSf15', 'ImSf15', 'ReSf17', 'ImSf17', 'ReSf19', 'ImSf19', 'ReSf21', 'ImSf21', 'ReSf23', 'ImSf23', 'ReSf25', 'ImSf25', 'ReSf29', 'ImSf29', 'ReSf33', 'ImSf33', 'ReSf37', 'ImSf37', 'ReSf43', 'ImSf43', 'ReSf49', 'ImSf49', 'ReSf57', 'ImSf57', 'ReSf69', 'ImSf69', 'ReSf83', 'ImSf83', 'ReSf101', 'ImSf101', 'ReSf127', 'ImSf127', 'ReSf165', 'ImSf165', 'ReSf237', 'ImSf237', 'ReSf399', 'ImSf399', 'ReSf1207', 'ImSf1207']

df = pd.read_csv(file_path, skiprows=4) # we skip the first four lines, because they are just metadata
df = df[fixed_features + labels]

validation_size = 0.1 # 90% training, 10% for validation

feature_scaler, label_r_scaler, label_i_scaler = compute_scalers(df, fixed_features, labels, validation_size, seed) # make sure we use the same seed, otherwise the two splits differ!
#dataset = ImpurityDataset(df, fixed_features, labels, device=device)
dataset = ImpurityDataset(df, fixed_features, labels, feature_scaler, label_r_scaler, label_i_scaler, device)  

indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=validation_size, random_state=seed)  # make sure we use the same seed, otherwise the two splits differ!

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [36]:
dataset.__getitem__(0)[0]

tensor([[ 1.6405, -0.2922, -0.1958,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.6405, -0.2922, -0.1958,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.6405, -0.2922, -0.1958,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 1.6405, -0.2922, -0.1958,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.6405, -0.2922, -0.1958,  ..., -1.8799,  0.0000,  0.0000],
        [ 1.6405, -0.2922, -0.1958,  ..., -1.8799,  0.0214, -1.8495]],
       device='cuda:0')

### Define the model

In [37]:
class PositionalEncodingL(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=27):
        super(PositionalEncodingL, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.positional_embedding = nn.Parameter(torch.zeros(max_len, d_model))
    
    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        batch_size, seq_len, d_model = x.shape
        
        # Use the first 'seq_len' positions from the positional_embedding
        # Expand the positional embeddings to match the batch size and add them to the input
        position_encoded = self.positional_embedding[:, :].unsqueeze(0).expand(batch_size, -1, -1)
        x = x + position_encoded
        return self.dropout(x)

In [38]:
class PositionalEncodingF(torch.nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=27):
        super(PositionalEncodingF, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [39]:
@dataclass
class ModelConfig:
    input_dim: int
    output_dim: int
    sequence_length: int
    
    d_model: int
    nhead: int
    num_layers: int
    dim_feedforward: int
    
    dropout: float
    activation: str
    bias: bool

class AutoregressiveTransformer(nn.Module):
    
    def __init__(self, config, device):
        super(AutoregressiveTransformer, self).__init__()

        self.config = config

        #self.input_projection = nn.Linear(config.input_dim, config.d_model)
        self.input_projection = nn.Linear(config.input_dim, config.d_model)
        
        self.positional_encoding = PositionalEncodingL(config.d_model, dropout=config.dropout, max_len=config.sequence_length)
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.d_model, 
            nhead=config.nhead, 
            dim_feedforward=config.dim_feedforward, 
            dropout=config.dropout,
            activation=config.activation, 
            batch_first=True, 
            norm_first=True, 
            bias=config.bias
        )
        
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.num_layers)
        
        self.output_layer = nn.Linear(config.d_model, config.output_dim)
        
        self.att_mask = self.generate_mask(config.sequence_length, device)
        
    def forward(self, x):        
        x = self.input_projection(x)        
        x = self.positional_encoding(x)
        
        output = self.transformer_decoder(x, x, tgt_mask=self.att_mask)
        output = self.output_layer(output)
        
        return output

    def generate_mask(self, sequence_length, device):
        mask = torch.zeros((sequence_length, sequence_length), device=device)
        mask = torch.triu(mask.fill_(float('-inf')), diagonal=1)
            
        return mask

### Initialize the model

In [40]:
config = ModelConfig(
    input_dim = 59,
    output_dim = 2,
    sequence_length = 27,
    
    d_model = 256,
    nhead = 4,
    num_layers = 2,
    dim_feedforward = 256 * 4,
    
    dropout = 0.1,
    activation = 'gelu',
    bias = True
)

model = AutoregressiveTransformer(config, device).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [41]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    
    for inputs, targets in train_loader:
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    return total_loss / len(train_loader)

In [42]:
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        
        for inputs, targets in val_loader:
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
    
    return total_loss / len(val_loader)

### Train the model

In [43]:
num_epochs = 200

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

C:\Users\Patri\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


KeyboardInterrupt: 

### Manual model validation

In [44]:
def validate_mape(model, val_loader, scaler_r, scaler_i, device, epsilon=1e-8):
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        
        for inputs, targets in val_loader:
            
            outputs = model(inputs)
            
            outputs = torch.tensor(np.array([reverse_tranform_output(o, scaler_r, scaler_i) for o in outputs]))
            targets = torch.tensor(np.array([reverse_tranform_output(t, scaler_r, scaler_i) for t in targets]))

            ape = torch.abs((targets - outputs) / (targets + epsilon))
            mape = torch.mean(ape) * 100
                    
            total_loss += mape.item()
    
    return total_loss / len(val_loader)

In [47]:
def reverse_tranform_output(data, scaler_r, scaler_i):
    data = data.cpu().numpy()
    
    r = scaler_r.inverse_transform(data[:, ::2].reshape(1, -1)).reshape(-1, 1)
    i = scaler_i.inverse_transform(data[:, 1::2].reshape(1, -1)).reshape(-1, 1)
            
    return np.concatenate((r, i), axis=1)

In [46]:
validate_mape(model, val_loader, label_r_scaler, label_i_scaler, device)

(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)
(27, 2)


26.98638406395912

### Sample from the model

In [324]:
def sample_from_model(model, input, sequence_length, fixed_feature_len, fixed_labels_len, max_features, device):
    model.eval()

    fixed_features = input[:fixed_feature_len]
    fixed_labels = input[fixed_feature_len:fixed_labels_len*2 + fixed_feature_len]
    
    assert len(fixed_labels) % 2 == 0
    provided_sequences = len(fixed_labels) // 2
    
    initial_input = torch.zeros(1, sequence_length, max_features)
    initial_input[0, 0, :fixed_feature_len] = fixed_features

    outputs = torch.zeros(sequence_length, 2, device = device)
    
    for i in range(1, provided_sequences + 1):
        labels = fixed_labels[(i-1)*2:2*i]
        initial_input[0, i, :2*i + fixed_feature_len] = initial_input[0, i-1, :2*i + fixed_feature_len]
        initial_input[0, i, 2*(i-1) + fixed_feature_len:2*i + fixed_feature_len] = labels
        outputs[i-1, :] = labels

    current_input = initial_input.to(device)

    
    with torch.no_grad():
    
        for i in range(fixed_labels_len + 1, sequence_length + 1):

            output = model(current_input)
            predictions = output[0, -1, -2:]

            outputs[i-1, :] = predictions
            
            if i == sequence_length:
                break
            
            position_to_insert = i * 2 + fixed_feature_len
            current_input[0, i, :position_to_insert-2] = current_input[0, i-1, :position_to_insert-2]
            current_input[0, i, position_to_insert-2:position_to_insert] = predictions

    return outputs

In [325]:
def convert(data):
    data = data.cpu().numpy()
            
    r = label_r_scaler.inverse_transform(data[::2].reshape(1, -1)).reshape(-1, 1)
    i = label_i_scaler.inverse_transform(data[1::2].reshape(1, -1)).reshape(-1, 1)
            
    return np.concatenate((r, i), axis=1)

In [326]:
def calculate_mape(true_values, predictions, epsilon=1e-8):
    true_values = np.array(true_values)
    predictions = np.array(predictions)
    
    mape = np.mean(np.abs((true_values - predictions) / (true_values + epsilon))) * 100
    return mape

In [335]:
n_samples = 100 #train_loader.dataset.dataset.n_samples 
sequence_length = 27
fixed_labels_len = 1
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2

mapes = []

for idx in range(n_samples):
    input = val_loader.dataset[idx][0][fixed_labels_len]
    output = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
    
    target = convert(val_loader.dataset[idx][1].reshape(-1))
    output = convert(output.reshape(-1))
    
    #print(output)
    #print(target)
    
    mapes.append(calculate_mape(target, output))

    if (idx+1) % 100 == 0:
        print(idx+1)

np.mean(mapes)

100


41.89146286994219

In [339]:
n_samples = 1000
sequence_length = 27
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2

total_mapes = []

for j in range(sequence_length):
    fixed_labels_len = j
    
    mapes = []
    
    for idx in range(n_samples):
        input = val_loader.dataset[idx][0][fixed_labels_len]
        output = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
        
        target = convert(val_loader.dataset[idx][1].reshape(-1))
        output = convert(output.reshape(-1))
        
        mapes.append(calculate_mape(target, output))
    
        if (idx+1) % 1000 == 0:
            print(idx+1)

    total_mapes.append(np.mean(mapes))

print(total_mapes)

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
[75.25655627772213, 50.31497908197343, 42.92279181443155, 38.64239037632942, 35.593322809785604, 33.093244361132385, 31.262917769886553, 29.305142525583506, 27.551480570062996, 25.754940614663063, 23.84885540297255, 22.35988664170727, 20.450180342793466, 18.69448260795325, 17.0201595001854, 15.231713772471995, 13.520954349264503, 11.926851070998236, 10.396295081684366, 8.944350967742503, 7.520265502529218, 6.1192999195773154, 4.780234487855341, 3.509259929810651, 2.300726163591025, 1.1484063471230912, 0.06026516933452512]


In [165]:
val_loader.dataset[0][1]

tensor([[-0.4675,  0.6046],
        [-0.6800,  0.8129],
        [-0.7581,  0.8897],
        [-0.7990,  0.9032],
        [-0.8237,  0.8701],
        [-0.8398,  0.8017],
        [-0.8506,  0.7071],
        [-0.8579,  0.5933],
        [-0.8626,  0.4660],
        [-0.8654,  0.3295],
        [-0.8669,  0.1872],
        [-0.8673,  0.0418],
        [-0.8668, -0.1046],
        [-0.8640, -0.3947],
        [-0.8597, -0.6742],
        [-0.8543, -0.9384],
        [-0.8455, -1.3009],
        [-0.8364, -1.6213],
        [-0.8247, -1.9857],
        [-0.8090, -2.4155],
        [-0.7940, -2.7782],
        [-0.7794, -3.0925],
        [-0.7650, -3.3635],
        [-0.7526, -3.5639],
        [-0.7416, -3.7089],
        [-0.7340, -3.7847],
        [-0.7300, -3.8136]], device='cuda:0')

In [22]:
val_loader.dataset[0][0][1]

tensor([ 4.0329, -0.9242,  0.5207, -0.2964,  0.1727, -0.7080,  1.2338, -0.4675,
         0.6046,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0')

In [414]:
idx = 0

model.eval()

input = val_loader.dataset[idx][0][0]
input2 = val_loader.dataset[idx][0][1]
target = val_loader.dataset[idx][1]

#context = torch.zeros(1, 27, 59, device = device)
#context[0, 0] = input

context = input.repeat(1, 27, 1)

for i in range(20):
    context[0, i] = val_loader.dataset[idx][0][i]


#context = input2.repeat(1, 27, 1)
#context[0, 1] = input2.repeat(1, 26, 1)
context.shape

#print(context)

#print(context[0][0])
#print(context[0][1])

with torch.no_grad():
    output = model(context)
    #print(output)
    #print(target)

# [2.8375185e-01 2.7025037e-04]
#[ 0.28389615 -0.00085208]

target = convert(target.reshape(-1))
output = convert(output.reshape(-1))

target, output

(array([[ 0.28389615, -0.00085208],
        [ 0.28396317, -0.00254925],
        [ 0.28409564, -0.00422577],
        [ 0.28429055, -0.00586867],
        [ 0.28454363, -0.00746612],
        [ 0.28484967, -0.00900773],
        [ 0.28520283, -0.01048475],
        [ 0.28559682, -0.01189018],
        [ 0.28602538, -0.01321872],
        [ 0.2864823 , -0.01446671],
        [ 0.28696173, -0.01563198],
        [ 0.28745827, -0.01671367],
        [ 0.287967  , -0.01771205],
        [ 0.28900412, -0.01946452],
        [ 0.29004437, -0.02090727],
        [ 0.29106668, -0.02206569],
        [ 0.29253575, -0.02333438],
        [ 0.29390275, -0.02413065],
        [ 0.29554573, -0.02462128],
        [ 0.29762554, -0.02451147],
        [ 0.2995346 , -0.02364393],
        [ 0.30134875, -0.02203338],
        [ 0.30310032, -0.01955435],
        [ 0.3045955 , -0.01640292],
        [ 0.3059083 , -0.01225413],
        [ 0.3068087 , -0.00762098],
        [ 0.30727825, -0.0025785 ]], dtype=float32),
 array([[ 0

In [413]:
idx = 0

model.eval()

input = val_loader.dataset[idx][0][start_with_n_labels]

original_input = input.clone()
    print(input)
    input = F.pad(input, (0, max_features - input.size(0)))
    print(input)
    context = input.repeat(1, sequence_length, 1)
    outputs = torch.tensor(np.zeros((sequence_length, 2)), device=device)

    #print(context.shape)
    
    with torch.no_grad():
    
        for i in range(start, sequence_length):

            #print(context[0][:7])
            output = model(context)
            
            prediction = output[:, -1:, :].reshape(-1) # get the last item

            #print(prediction)
            outputs[i] = prediction

            if i+1 == sequence_length: 
                break
            
            # Append the next item to the generated sequence
            test = torch.cat((original_input, outputs[:i+1].reshape(-1)), dim=0)
            #print(test.shape)
            context[0, i+1, :] = F.pad(test, (0, max_features - test.size(0)))

IndentationError: unexpected indent (94715631.py, line 8)

In [289]:
target = convert(train_loader.dataset[1][1].reshape(-1))
target

array([[ 0.28438747, -0.03047266],
       [ 0.2913718 , -0.05072586],
       [ 0.30114108, -0.05112787],
       [ 0.30832806, -0.04617152],
       [ 0.31295744, -0.04065794],
       [ 0.31593782, -0.0357836 ],
       [ 0.31791824, -0.03171095],
       [ 0.31928274, -0.02834773],
       [ 0.3202551 , -0.02556159],
       [ 0.3209689 , -0.02323411],
       [ 0.3215065 , -0.02127034],
       [ 0.32192057, -0.0195967 ],
       [ 0.32224572, -0.01815656],
       [ 0.3227158 , -0.01581195],
       [ 0.32303217, -0.01399025],
       [ 0.3232548 , -0.01253754],
       [ 0.32348233, -0.0108408 ],
       [ 0.32363304, -0.00954406],
       [ 0.3237658 , -0.00822783],
       [ 0.323886  , -0.00681435],
       [ 0.32396623, -0.00567463],
       [ 0.3240247 , -0.00466912],
       [ 0.32406953, -0.00371678],
       [ 0.32410103, -0.00286272],
       [ 0.3241247 , -0.00199403],
       [ 0.3241391 , -0.00118479],
       [ 0.3241461 , -0.00039172]], dtype=float32)

In [290]:
output = convert(output[7:])
output

AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [5]:
calculate_mape(target, output)

NameError: name 'calculate_mape' is not defined

### Todos and other stuff

In [397]:
def validate_manual(model, val_loader, idx, seq, criterion, device):
    model.eval()
    #total_loss = 0.0

    with torch.no_grad():
        
        for inputs, targets in val_loader:

            print(inputs[0].reshape(1, 27, 59).shape)
            
            outputs = model(inputs[0].reshape(1, 27, 59))
            ##loss = criterion(outputs, targets)

            #target = targets[idx]
            #out = outputs[idx]
            
            target = convert(targets, idx)
            out = convert(outputs, idx)

            actual = out
            exptected = target

            print(actual[seq])
            print(exptected[seq])
            
            #total_loss += loss.item()

            break
    
    return actual, exptected, 0

In [398]:
def convert(targets, idx):
    target_seq = targets.cpu().numpy()
            
    r = label_r_scaler.inverse_transform(target_seq[idx, :, ::2].reshape(1, -1)).reshape(-1, 1)
    i = label_i_scaler.inverse_transform(target_seq[idx, :, 1::2].reshape(1, -1)).reshape(-1, 1)
            
    return np.concatenate((r, i), axis=1)

In [399]:
def calculate_mape(true_values, predictions, epsilon=1e-8):
    # Ensure true_values and predictions are numpy arrays
    true_values = np.array(true_values)
    predictions = np.array(predictions)
    
    # Calculate MAPE
    mape = np.mean(np.abs((true_values - predictions) / (true_values + epsilon))) * 100
    return mape

In [400]:
idx = 0
seq = 0
actual, exptected, loss = validate_manual(model, val_loader, idx, seq, criterion, device)
loss, calculate_mape(exptected[seq][1], actual[seq][1])

torch.Size([1, 27, 59])
[2.8375185e-01 2.7025037e-04]
[ 0.28389615 -0.00085208]


(0, 131.71827010002053)

In [14]:
def sample_from_model(model, input, sequence_length, device):
    model.eval()
    with torch.no_grad():
    
        for _ in range(sequence_length):
            output = model(input)
            print(output)
            next_item = output[:, -1:, :].repeat(1, sequence_length, 1)
    
            print(next_item)
    
            # Append the next item to the generated sequence
            sample = torch.cat((sample, next_item), dim=2)
            print(sample[0][0])
        
        return sample

In [18]:
sequence_length = 27

input = torch.tensor(np.zeros((1, sequence_length, 59), dtype='f'), device=device)

sample = train_loader.dataset[1][0][0]
label = train_loader.dataset[1][1][0]

print(label)

input[0, 0] = sample
input.shape

sample_from_model(model, input, sequence_length, device)

#sample = sample.repeat(sequence_length, 1).unsqueeze(0)
#sample, label

tensor([ 0.2844, -0.0305], device='cuda:0')
tensor([[[-0.0002,  0.0046],
         [ 0.0017,  0.0091],
         [ 0.0020,  0.0091],
         [ 0.0021,  0.0091],
         [ 0.0022,  0.0091],
         [ 0.0022,  0.0091],
         [ 0.0022,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0091],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090],
         [ 0.0023,  0.0090]]], device='cuda:0')
tensor([[[0.0023, 0.0090],
         [0.0023, 0.0090],
         [0.0023, 0.0090],
         [0.0023, 0.0090],
         [0.0023, 0.0090],
         [0.0023, 0

UnboundLocalError: cannot access local variable 'sample' where it is not associated with a value

In [136]:
with torch.no_grad():

    for _ in range(sequence_length):
        output = model(input)
        print(output)
        next_item = output[:, -1:, :].repeat(1, sequence_length, 1)

        print(next_item)

        # Append the next item to the generated sequence
        sample = torch.cat((sample, next_item), dim=2)
        print(sample[0][0])

tensor([[[ 0.5402, -0.1061],
         [ 0.0693,  0.0009],
         [ 0.0704,  0.0061],
         [ 0.0706,  0.0067],
         [ 0.0711,  0.0064],
         [ 0.0728,  0.0051],
         [ 0.0737,  0.0050],
         [ 0.0733,  0.0046],
         [ 0.0707,  0.0041],
         [ 0.0728,  0.0033],
         [ 0.0723,  0.0032],
         [ 0.0749,  0.0023],
         [ 0.0775,  0.0009],
         [ 0.0788,  0.0010],
         [ 0.0794,  0.0011],
         [ 0.0794,  0.0015],
         [ 0.0799,  0.0017],
         [ 0.0803,  0.0018],
         [ 0.0823,  0.0019],
         [ 0.0821,  0.0027],
         [ 0.0801,  0.0032],
         [ 0.0813,  0.0030],
         [ 0.0824,  0.0020],
         [ 0.0824,  0.0021],
         [ 0.0823,  0.0021],
         [ 0.0840,  0.0013],
         [ 0.0852,  0.0007]]], device='cuda:0')
tensor([[[0.0852, 0.0007],
         [0.0852, 0.0007],
         [0.0852, 0.0007],
         [0.0852, 0.0007],
         [0.0852, 0.0007],
         [0.0852, 0.0007],
         [0.0852, 0.0007],
         

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 2)

In [68]:
output

NameError: name 'output' is not defined

In [15]:
 0.151 * 1e-3

0.000151

In [17]:
0.046 * 1e-3

4.6e-05